In [1]:
import json
import os
import cv2
from pathlib import Path
from PIL import Image
import pandas as pd
from pandas import json_normalize
import random
import re

import numpy as np
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

# Load the SDK
import lyft_dataset_sdk
from lyft_dataset_sdk.lyftdataset import LyftDataset, LyftDatasetExplorer, Quaternion, view_points
from lyft_dataset_sdk.utils.data_classes import LidarPointCloud
from lyft_dataset_sdk.utils.geometry_utils import BoxVisibility, box_in_image, view_points

In [2]:
data_path = Path(r"S:\MADS\Capstone\3d-object-detection-for-autonomous-vehicles")
json_path = r"S:\MADS\Capstone\3d-object-detection-for-autonomous-vehicles\data"
lyftdata = LyftDataset(data_path=data_path, json_path=json_path, verbose=True)

9 category,
18 attribute,
4 visibility,
18421 instance,
10 sensor,
148 calibrated_sensor,
177789 ego_pose,
180 log,
180 scene,
22680 sample,
189504 sample_data,
638179 sample_annotation,
1 map,
Done loading in 8.3 seconds.
Reverse indexing ...
Done reverse indexing in 2.6 seconds.


## Begin

In [14]:
#In order to find the image files that contain our moto_anns, we must do the following:
#Accumulate the superset of Samples that contain all of our moto_annotations
def img_class_selector(class_name, box_visibility = BoxVisibility.ALL):
    """
    There wasn't a super clean way in the API to select image files that contain annotations from x class. I adapted the method they use in their .render_annotation() method.

    Returns tuples of (file_path, Box object, camera_intrinsic, sample_token, cam)
    """

    annos_class = lyftdata.sample_annotation
    class_anns = [d for d in annos_class if d["category_name"] == class_name]
    sample_class_tokens = set()
    for ann in class_anns:
        sample_class_tokens.add(ann["sample_token"])

    class_images_boxes = []
    for sample_token in sample_class_tokens:
        sample_record = lyftdata.get("sample", sample_token)
        sample_anns = sample_record["anns"]
        cams = [key for key in sample_record["data"].keys() if "CAM" in key]
        
        #We unfortunately do have to iterate through each annotation that exists within the sample_record. There typically aren't more than 20-30 or so.
        for ann in sample_anns:
            # Figure out which camera the object is fully visible in (this may return nothing)
            for cam in cams:
                path, box, camera_intrinsic = lyftdata.get_sample_data(sample_record["data"][cam], box_vis_level=box_visibility, selected_anntokens=[ann])
                if box:
                    if box[0].name == class_name:
                        class_images_boxes.append((str(path), box[0], camera_intrinsic, sample_token, cam, ann))

    return class_images_boxes

In [29]:
def crop_select_images(img_class_selector_output, output_path, box_area_threshold = 2000):
    path, box, camera_intrinsic, sample_token, cam, ann = img_class_selector_output
    box_coords = view_points(box.corners(), view = camera_intrinsic, normalize=True)[:2, :]
    
    # Calculate bounding box coordinates
    x_min = np.min(box_coords[0])
    y_min = np.min(box_coords[1])
    x_max = np.max(box_coords[0])
    y_max = np.max(box_coords[1])

    # Calculate the area of the bounding box
    box_area = (x_max - x_min) * (y_max - y_min)

    if box_area > box_area_threshold:
        im = Image.open(path)

        match = re.search(r'images\\(.+)', path)

        if match:
            file_name = match.group(1)
        else:
            print("Pattern not found in the path", path)
            pass

        # Crop the image
        # Pillow uses a system of (left, upper, right, lower)
        cropped_im = im.crop((x_min, y_min, x_max, y_max))

        # Check if the directory exists, if not, create it
        Path(output_path).mkdir(parents=True, exist_ok=True)

        #Save the image
        cropped_im.save(os.path.join(output_path, "cropped_" + file_name[:-5] + "_" + ann + file_name[-5:]))

In [30]:
categories = [lyftdata.category[i]["name"] for i in range(len(lyftdata.category))]
categories

['car',
 'pedestrian',
 'animal',
 'other_vehicle',
 'bus',
 'motorcycle',
 'truck',
 'emergency_vehicle',
 'bicycle']

In [31]:
for cat in categories:

    #We can skip car because we know we aren't doing this one, plus its by far the largest and most timely.
    if cat != "car":
        #Obtain all of the annotations and their corresponding data via image_class_selector
        cat_images_boxes = img_class_selector(cat)
        img_output_path = os.path.join(r"S:\MADS\Capstone\3d-object-detection-for-autonomous-vehicles\images", categories[0]+"_cropped")

        #Do the actual cropping and write cropped images to disk
        for cat_images_box in cat_images_boxes:
            crop_select_images(cat_images_box, output_path=img_output_path, box_area_threshold=5000)


        #Since we went through the effort of running img_class_selector, lets write this to disk for each category so we don't have to run every time.

        # Convert NumPy arrays to lists (if any)
        # For example, if 'camera_intrinsic' is a NumPy array:
        cat_images_boxes['motorcycle'][0]['camera_intrinsic'] = cat_images_boxes['motorcycle'][0]['camera_intrinsic'].tolist()
        data_output_path = os.path.join(r"S:\MADS\Capstone\3d-object-detection-for-autonomous-vehicles\sterny_data", categories[0], ".json")
        
        # Check if the directory exists, if not, create it
        output_dir = os.path.dirname(data_output_path)
        Path(output_dir).mkdir(parents=True, exist_ok=True)

        # Write to a JSON file at the specified path
        with open(data_output_path, 'w') as file:
            json.dump(cat_images_boxes, file, indent=4)

TypeError: Object of type Box is not JSON serializable

In [ ]:
output_path = r"S:\MADS\Capstone\3d-object-detection-for-autonomous-vehicles\images\moto_cropped"
for moto_img_box in motorcycle_images_boxes:
    crop_select_images(moto_img_box, output_path=output_path, box_area_threshold=5000)

In [13]:
write_path = r"S:\MADS\Capstone\3d-object-detection-for-autonomous-vehicles\sterny_data"

# Write to a JSON file at the specified path
with open(write_path, 'w') as file:
    json.dump(motorcycle_images_boxes, file, indent=4)

PermissionError: [Errno 13] Permission denied: 'S:\\MADS\\Capstone\\3d-object-detection-for-autonomous-vehicles\\sterny_data'